# Char level transformer for Machine translation
To do translation, we'll use sequence-to-sequence model: an encoder encode the input and decoder using both the encode and raw output to predict. 

This is similar to char-transformer for language modeling, but we will add an encoder, and in place of top-k output when we generate text we'll use top-1 beam search output.

<img src='images/transformer.png' width=400>

<a href="https://www.kaggle.com/hungnm/englishvietnamese-translation">English-Vietnamese HungMN Kaggle</a>

<a href="https://github.com/vietai/SAT">English-Vietnamese dataset VietAI SAT</a>

<a href="https://nlp.stanford.edu/projects/nmt/">English-Vietnamese dataset IWSLT 15</a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/MyDrive/Colab Notebooks/language_translation/CharTransformerTranslation'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/language_translation/CharTransformerTranslation


In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
# Load saved data
from utils.utils import *
from utils.pre_processing import *

import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
class CharDataset(Dataset):
    '''
    Dataset is a iterable that returns input and target sentence. 
    It adds <sos> at the begining, and <eos> at the end, 
    and filling in <pad> if sentence length is less than pre-defined value.
    '''
    
    def __init__(self, x, y, sequence_len, encoder=None):
        
        # data in the type of pairs of sentence
        data = ''.join(x+y)
        chars = ['<pad>'] +['<sos>'] + ['<eos>'] + sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)

        print('data has %d characters, %d unique chars, %d sentences.' % (data_size, len(chars), len(x)))
        print('sentence length nine_nine_percentile: %d' % (sequence_len))
        
        self.x, self.y = x, y
        # vocab combining both x and y chars
        self.ch2i = {ch:i for i,ch in enumerate(chars)}
        self.i2ch = {i:ch for i,ch in enumerate(chars)}
        self.vocab_size = vocab_size
        
        self.sequence_len = sequence_len
        self.encoder=encoder
    
    def __len__(self):
        return len(self.x) # len x = y
    
    def __getitem__(self, idx):
        
        indx = self.padding([self.ch2i[ch] for ch in self.x[idx]] + [self.ch2i['<eos>']])
        indy = [self.ch2i['<sos>']] + self.padding([self.ch2i[ch] for ch in self.y[idx]] + [self.ch2i['<eos>']])

        x = torch.tensor(indx, dtype=torch.long)
        y = torch.tensor(indy, dtype=torch.long)

        return x,y
                                                                                                                               
    def padding(self, string):
        if len(string)<self.sequence_len:
            string =  string + [0]*(self.sequence_len - len(string))
        else:
            string = string[:self.sequence_len -1] + [self.ch2i['<eos>']]
                   
        return string

In [4]:
sequence_len = 128
min_len = 5

In [6]:
paths = [
        # "data/vietaisat/",
         ("data/hungnm/vi.txt", "data/hungnm/en.txt"),
         ("data/iwslt15/vi.txt", "data/iwslt15/en.txt")
         ]
X, Y = load_data(paths, min_len, sequence_len)

path = "data/cleaned/"

# save to pre-defined path
pickle(path+"X", X)
pickle(path+"Y", Y)

# delete after save
del X
del Y

processing path:  ('data/hungnm/vi.txt', 'data/hungnm/en.txt')
clean text using re.sub
data size before x:254090, y:254090
remove sentence base on min max length


100%|██████████| 254090/254090 [09:05<00:00, 465.82it/s] 


data size after x:243851, y:243851
Some last sentences
bạn đã ở đám cưới của tôi | You were at my wedding
tại sao bạn không quan tâm? | Why aren't you interested?
tom đã vay 300 từ mary | Tom borrowed 300 from Mary
Tom ở lại boston trong ba tháng. | Tom stayed in Boston for three months.


processing path:  ('data/iwslt15/vi.txt', 'data/iwslt15/en.txt')
clean text using re.sub
data size before x:133318, y:133318
remove sentence base on min max length


100%|██████████| 133318/133318 [01:49<00:00, 1214.76it/s]


data size after x:94751, y:94751
Some last sentences
Nó là do con người và có thể ngăn chặn và diệt trừ bởi hành động của con người .  | It s manmade and can be overcome and eradicated by the actions of human beings . 
Mối liên kết chung trong những việc chúng tôi làm chỉ có 1 , đó là nghèo đói . | The common link between all the work we ve had to do is one thing , and that s poverty .
Những vấn đề này không phân biệt lãnh thổ , màu da hay tôn giáo . Không hề . | None of those are limited by geography , by skin color or by religion . None of them .
Vấn đề là điều kiện sống nghèo khổ , nhà cửa tồi tàn , và côn trùng gây bệnh . | The problem poor living environment , poor housing , and the bugs that do people harm .




In [5]:
# load cleaned data
path = "data/cleaned/"

# load data which is list of sentences, pickle here is just to save storage space
X = pickle(path+"X")
Y = pickle(path+"Y")

for i in range(-1,-5,-1):
    print(X[i],'|',Y[i])

assert(len(X)==len(Y))
len(X), len(Y)

It s manmade and can be overcome and eradicated by the actions of human beings .  | Nó là do con người và có thể ngăn chặn và diệt trừ bởi hành động của con người . 
The common link between all the work we ve had to do is one thing , and that s poverty . | Mối liên kết chung trong những việc chúng tôi làm chỉ có 1 , đó là nghèo đói .
None of those are limited by geography , by skin color or by religion . None of them . | Những vấn đề này không phân biệt lãnh thổ , màu da hay tôn giáo . Không hề .
The problem poor living environment , poor housing , and the bugs that do people harm . | Vấn đề là điều kiện sống nghèo khổ , nhà cửa tồi tàn , và côn trùng gây bệnh .


(338602, 338602)

In [6]:
# load to dataset
dataset = CharDataset(X, Y, sequence_len=sequence_len)

print('sample tensors ', next(iter(dataset)))
# ong strength of char transformer is we can combine vocab and 
# encoder all the languages
print("vocab: ", dataset.ch2i)

data has 29027595 characters, 184 unique chars, 338602 sentences.
sentence length nine_nine_percentile: 128
sample tensors  (tensor([ 69,  54,  59,   3,  67,  66,  54,   3,  57,  93,  59,  52,   3, 103,
        129,  65,   3,  59,  52, 110, 160,  54,   3,  62,  66,  89,  65,   3,
         63,  85,  48,   3,  65,  63,  60,  59,  52,   3,  65, 168,   3,  48,
         53, 153,  54,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]), tensor([ 1, 34, 57, 50, 46, 64, 50,  3, 61, 66, 65,  3, 65, 53, 50,  3, 49, 66,
        64, 65, 61, 46, 59,  3, 54, 59,  3, 65, 53, 50,  3, 47, 63, 60, 6

# Building model

In [7]:
from model.encode_decode_transformer import Transformer, TransformerConfig
from utils.trainer import Trainer, TrainerConfig
tconfig = TrainerConfig(max_epochs=1, batch_size=32, learning_rate=6e-4, grad_norm_clip=1.0, device='cuda',
                       lr_decay=True, warmup_tokens=5000, ckpt_n_print_iter=4000, ckpt_path='checkpoint/transformer_vn_en_char')

mconfig = TransformerConfig(vocab_size=dataset.vocab_size, sequence_len=dataset.sequence_len, embed_dim=256,
                           n_block=12, n_head=8, device=tconfig.device)

In [8]:
model = Transformer(mconfig)

In [9]:
sentences = ["hôm nay trông bạn thật đẹp!",
           "đưa tôi một chai nước, tôi khát khô cổ rồi.",
           "thời tiết hôm nay thật đẹp!",
           "bạn đã ăn sáng chưa?",
           "chúng ta sẽ khởi hành vào rạng sáng mai."
            ]
trainer = Trainer(model, dataset, tconfig, test_dataset=sentences, collate=None)

In [ ]:
# load pre-trained weights
model.load_state_dict(pickle(tconfig.ckpt_path)) # load

In [10]:
# for around 300k sentences pair, the configured transformer results in 1 hour training for 1 epoch
trainer.train()

epoch: 1 | train loss: 900.54199  | lr: 3.840000e-06:   0%|          | 0/10581 [00:01<?, ?it/s]

[]


epoch: 1 | train loss: 1.20040  | lr: 4.201730e-04:  38%|███▊      | 4000/10581 [23:15<37:12,  2.95it/s]

["You're going to be the car, you're getting.", "My father, I wasn't the car, but she wasn't the country.", "My father, I'm going to get there!", 'Did you get the parents?', "We'll be better in Boston in Boston."]


epoch: 1 | train loss: 0.96452  | lr: 8.620048e-05:  76%|███████▌  | 8000/10581 [46:24<14:39,  2.94it/s]

['When did you go to the station!', "My father was a little, but I don't want to.", 'Who knows when he was?', 'Have you eaten your house?', "We're going to be here in the world."]


epoch: 1 | train loss: 0.93882  | lr: 6.000000e-05: 100%|██████████| 10581/10581 [1:01:28<00:00,  2.87it/s]


In [14]:
# inference som examples
samples = [
            "How is the weather today?",
            "Hôm nay cái áo bạn mặc trông thật đẹp, nó bao tiền vậy?",
           "Nhà tôi có 1 con zombie trà sữa",
           "nó đáng yêu nhưng rất sợ sấm và bóng tối",
           "Hôm nay nhìn Amy không khác gì tranh vẽ",
           "dù Amy chưa ăn sáng",
           "Chúng ta sẽ khởi hành vào rạng sáng mai, hãy chuẩn bị kỹ."
          ]
result = model.generate_output(samples, dataset, top_k=5, print_process=True)
for i in range(len(samples)):
  print(samples[i], " | ", result[i])

100%|██████████| 7/7 [00:13<00:00,  1.94s/it]


In [13]:
# benchmarking test data using bleu score
path = "data/iwslt15/"
X = open(path+"tst2013.en.txt", encoding='utf-8').read().split("\n")
Y = open(path+"tst2013.vi.txt", encoding='utf-8').read().split("\n")
X,Y = pre_processing(X, Y, min_length=min_len, max_length=sequence_len) # remove sentence less than 4 characters

result = model.generate_output(X, dataset, top_k=5, print_process=True)

score, references, candidates = bleu_score(en, result)
print("bleu score: ", score)

# print example translations
# for i in range(len(vi)):
#     print(vi[i], " | ", en[i], " | ", result[i])


clean text using re.sub
data size before x:1269, y:1269
remove sentence base on min max length


100%|██████████| 1269/1269 [00:00<00:00, 123823.93it/s]


data size after x:851, y:851
Some last sentences
They knew their image would be seen by you out in the world . | Họ biết hình của họ sẽ được xem bởi những người ở ngoài kia , như bạn
I want to shine a light on slavery . | Tôi muốn đưa nô lệ ra ánh sáng .
On top of that , Manuru has tuberculosis , yet he s still forced to work day in and day out in that mine shaft . | Trên hết , Manuru bị bệnh lao , nhưng vẫn bị bắt làm việc ngày qua ngày tại hầm mỏ đó .
When his uncle died , Manuru inherited his uncle s debt , which further forced him into being enslaved in the mines . | Khi người chú mất , Manuru kế thừa món nợ của chú , và bị buộc làm nô lệ trong khu mỏ này lâu hơn nữa .




100%|██████████| 851/851 [28:14<00:00,  1.99s/it]


bleu score:  2.908047833407676
Tôi đã rất tự hào về đất nước tôi .  |  And I was very proud .  |  I was very interested with my father .
Gia đình của tôi không nghèo , và bản thân tôi thì chưa từng phải chịu đói .  |  My family was not poor , and myself , I had never experienced hunger .  |  My father is not my favorite , and I have to do that , but I have to do it .
Nhưng vào một ngày của năm 1995 , mẹ tôi mang về nhà một lá thư từ một người chị em cùng chỗ làm với mẹ .  |  But one day , in 1995 , my mom brought home a letter from a coworker s sister .  |  But this is one of the five years ago , one of these things that I wanted to do something in the world .
Tất cả cùng nằm trên sàn , và cơ thể chúng tôi yếu đến có thể cảm thấy như cái chết đang đến rất gần .  |  We are lying on the floor together , and our bodies are so weak we are ready to die .   |  All of these people were , we could have been trying to talk about those people .
Tôi đã bị sốc .  |  I was so shocked .  |  I have b